In [1]:
import sys
import os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import matplotlib.patches as patches
from shapely.geometry import Point
from shapely.geometry import MultiPoint
from shapely.geometry import MultiPolygon
from shapely.geometry import Polygon
from shapely.geometry import box
from descartes.patch import PolygonPatch
import ipywidgets as widgets
import ipywidgets.widgets.interaction
#from ipywidgets import interactive, fixed
from ipywidgets import *
import random
import pandas as pd

import re

from shapely import geometry
import pickle
import time

#from ipyparallel import Client

from shapely import affinity
from shapely.geometry import LineString
import warnings
warnings.simplefilter('ignore', np.RankWarning)


import gc
gc.enable()

In [2]:
#os.chdir('/home/charly/Lab/Projects/uJ/local/experiments/HT-Sine2hr/')
#os.chdir('/home/esb/Projects/uJ/uJ_master/experiments/HT-Sine2hr/')

In [3]:
pwd

'/home/esb/Projects/uJ/local/experiments/HT-LB'

In [13]:
#sys.path.insert(0, '/Users/ESB/SYNC_RPM/RPM_Work/uJ_master/current/uJ_src/python/')
#sys.path.insert(0, '/home/esb/Projects/uJ/uJ_master/current/uJ_src/python/')
#sys.path.insert(0, '/home/charly/Lab/Projects/uJ/uJ_master/current/uJ_src/python/')
sys.path.insert(0, '/home/esb/Projects/uJ/local/current/uJ_src_pHT/python/')
sys.path.insert(0, '/home/charly/Lab/Projects/uJ/local/current/uJ_src_PHT/python/')

#from readroi import *
from DataManagers import *
from DataStructs import *
from TrackingFunctions import *
from PlottingFunctions import *
from AnalysisFunctions import *

> Analysis Functions loaded


## Experiment Parameters

In [5]:
rootDir="../../data_sample/HT-LB/"
expeLabel="HT-LB"
trap_list="xy02,xy03,xy04,xy05,xy06,xy10".split(",")  
trap_list="xy03,xy03-1".split(",")  


data_type="normData"
#Input files



figxlim=1000
figylim=512
flims=(figxlim,figylim)



for trap in trap_list:
    #Input files
    
    dirNameDATA=rootDir+'data/'+data_type+'/'+trap+"/"
    
    #Output
    dirNameCELLS=rootDir+'data_cells/'+trap+'/'+data_type+'/'
   

    dirNameTRACKEDCELLS=rootDir+'data_cells_tracked/'+trap+'/'+data_type+'/'
    


    dirNameTRACKEDCELLS_IMGS=rootDir+'figures/traked_cells/'+trap+'/'
    if not os.path.exists(dirNameTRACKEDCELLS_IMGS):
        print("making dir figures...")
        os.makedirs(dirNameTRACKEDCELLS_IMGS)
   
    dirNameDATALINEAGES=rootDir+'data/lineages/'
      

#toFile=False
toFile=True
    

frame2min=5

notcaptured_start=866
notcaptured_end=866


frame_experiment_start=1;
frame_signal_start=int(865-(frame_experiment_start-1)/2)
frame_signal_end=int(865-(frame_experiment_start-1)/2)
frame_experiment_end=int(865-(frame_experiment_start-1)/2)

t_experiment_start=0
t_signal_start=(frame_signal_start-frame_experiment_start)*frame2min  
t_signal_end=(frame_signal_end-frame_experiment_start)*frame2min
t_experiment_end=(frame_experiment_end-frame_experiment_start)*frame2min


frames=range(int(frame_experiment_start),int(frame_experiment_end+1))

times=[(this_frame-frame_experiment_start)*frame2min for this_frame in frames]
times_hour=[(this_frame-frame_experiment_start)*frame2min/60 for this_frame in frames]
#print("\nMinutes:  %s"%times)
#print("\nHours:  [%s,%s]"%(times_hour[0],times_hour[-1]))


print("\nFrames:  %s\t %s->%s hours"%(frames,times_hour[0],times_hour[-1]))
print("t_experiment_start=%s\t(%sh)"%(t_experiment_start, t_experiment_start/60))
print("t_experiment_end=%s\t(%sh)"%(t_experiment_end, t_experiment_end/60))



#print(dataPath)
#if not os.path.exists(figurePathDM) and toFile:


Frames:  range(1, 866)	 0.0->72.0 hours
t_experiment_start=0	(0.0h)
t_experiment_end=4320	(72.0h)


 ## Load tracked cells

In [7]:

max_frames=-1
#max_frames=10

trap_tracked_cells=[]
for trap in trap_list:
    print(trap)
    dirNameTRACKEDCELLS=rootDir+'data_cells_tracked/'+trap+"/"+data_type+"/"
    
    tracked_cells=load_cells(dirNameTRACKEDCELLS,max_frames)
    print(len(tracked_cells))
    trap_tracked_cells.append(tracked_cells)

xy03
864: Loading HT-LB_xy03_865.pkl
865
xy03-1
864: Loading HT-LB_xy03-1_865.pkl
865


## Load lineages

In [8]:
def load_data_lineages(fileName):
    df_lineages = pd.read_csv(fileName,dtype=object)
    df_lineages["frame"] = pd.to_numeric(df_lineages["frame"])
    #df_lineages = pd.read_csv(fileName)
    print('Loading %s lineages from %s'%(len(df_lineages.lineageID.unique()), fileName))
    return df_lineages


trap_df_lineages=[]
trap_df_lineages_long=[]
for i,trap in enumerate(trap_list):
    #cell_lineages=trap_lineages[i]
    dirNameDATALINEAGES=rootDir+'data/lineages/'
    
    lineagesDataFile='%s%s_%s_lineages_all.csv'%(dirNameDATALINEAGES, expeLabel, trap)
    df_lineages=load_data_lineages(lineagesDataFile)
    trap_df_lineages.append(df_lineages)
    list_frames=df_lineages.frame.unique()
    lineagesDataFile='%s%s_%s_lineages_analysis.csv'%(dirNameDATALINEAGES, expeLabel, trap)
    df_lineages_long=load_data_lineages(lineagesDataFile)
    trap_df_lineages_long.append(df_lineages_long)
list_frames=[int(x) for x in list_frames]
print(len(list_frames))

Loading 168 lineages from ../../data_sample/HT-LB/data/lineages/HT-LB_xy03_lineages_all.csv
Loading 8 lineages from ../../data_sample/HT-LB/data/lineages/HT-LB_xy03_lineages_analysis.csv
Loading 15 lineages from ../../data_sample/HT-LB/data/lineages/HT-LB_xy03-1_lineages_all.csv
Loading 1 lineages from ../../data_sample/HT-LB/data/lineages/HT-LB_xy03-1_lineages_analysis.csv
865


## Select final & initial cells

In [ ]:
%matplotlib notebook

In [11]:
%matplotlib ipympl
#%matplotlib widgets

 #if plot does not show,uncomment this line run, comment and re-run
def onclickSelectMulti(event):

       #global max_frames

       #First find if Point is inside a roi
    this_point = Point(event.xdata,event.ydata)
    #if(event.y)
    
    titleA=event.inaxes.get_title()
    title=titleA.split(" ")[0]
    tindex2=int(titleA.split(" ")[1])
    
    tindex=trap_list.index(title)
    to_track_polys0=trap_to_track_polys0[tindex]
    
    to_track_polysf=trap_to_track_polysf[tindex]
    ax=axs[tindex,tindex2]
   # print(tindex,tindex2)
    if(tindex2==0):
        
        for select_index, select_poly in enumerate(to_track_polys0):
            in_roi=select_poly.contains(this_point)
            if in_roi:
                select_x,select_y = select_poly.exterior.xy     
                ax.plot(select_x, select_y, color='#6699cc', alpha=0.7, linewidth=2, solid_capstyle='round', zorder=3)
                #ax.axis('off')
                cellID=trap_tracked_cells[tindex][0][select_index]['cellID']
                trap_toTrack_cellID0[tindex].append(cellID)
                #trap_toTrack_roiIndex0[tindex].append(select_index)
                #trap_toTrack_roiFrame0[tindex].append(to_track_frame)
                #trap_toTrack_roiFrame0[tindex].append(1)
                #trap_toTrack_trapIndex

                break
    if(tindex2==1):
        
        for select_index, select_poly in enumerate(to_track_polysf):
            in_roi=select_poly.contains(this_point)
            if in_roi:
                select_x,select_y = select_poly.exterior.xy     
                ax.plot(select_x, select_y, color='#6699cc', alpha=0.7, linewidth=2, solid_capstyle='round', zorder=3)
                #ax.axis('off')
                cellID=trap_tracked_cells[tindex][final_frame][select_index]['cellID']
                trap_toTrack_cellIDf[tindex].append(cellID)
                #trap_toTrack_roiIndexf[tindex].append(select_index)
                #trap_toTrack_roiFramef[tindex].append(to_track_frame)
                #trap_toTrack_roiFramef[tindex].append(final_frame)
                #trap_toTrack_trapIndex

                break


fig,axs=plt.subplots(len(trap_tracked_cells),2,figsize=(15,15))
trap_to_track_polys0=[]
trap_to_track_polysf=[]

trap_toTrack_cellID0=[]
trap_toTrack_cellIDf=[]

for ti,t_cells in enumerate(trap_tracked_cells):
    cells=t_cells
    ini_frame=1
    final_frame=len(cells)-1
    #to_track_frame=14
   # global toTrack_roiIndex
    trap_toTrack_cellID0.append([])
    trap_toTrack_cellIDf.append([])
    



    
    #fig=plt.figure()
    ax = axs[ti,0]
    ax.set_xlim(0,figxlim)
    ax.set_ylim(0,figylim)
    #ax.clear()
    to_track_polys0=[]
    to_track_cellIDs0=[]
    to_track_polysf=[]
    to_track_cellIDsf=[]
        

    for this_cell in cells[ini_frame-1]:
        this_poly=this_cell['roiPoly']
        this_cellID=this_cell['cellID']
        to_track_polys0.append(this_poly)
        to_track_cellIDs0.append(this_cellID)
        patch = PolygonPatch(this_poly, facecolor=[0.75,0.75,0.75], edgecolor=[0,0,0], alpha=0.7, zorder=2)
        ax.add_patch(patch)
       #ax.plot(x=100,y=100,c='r*')
    #ax.axis('off')
    trap_to_track_polys0.append(to_track_polys0)
    #trap_toTrack_cellID0.append(to_track_cellIDs0)
    ax.set_title("%s %s"%(trap_list[ti],0))
    ax.set_aspect('equal')
    ax.invert_yaxis()
    
    
    ax = axs[ti,1]
    ax.set_xlim(0,figxlim)
    ax.set_ylim(0,figylim)
    #ax.clear()
    to_track_polysf=[]

    for this_cell in cells[final_frame]:
        this_poly=this_cell['roiPoly']
        this_cellID=this_cell['cellID']
        to_track_polysf.append(this_poly)
        to_track_cellIDsf.append(this_cellID)
        
        patch = PolygonPatch(this_poly, facecolor=[0.75,0.75,0.75], edgecolor=[0,0,0], alpha=0.7, zorder=2)
        ax.add_patch(patch)
       #ax.plot(x=100,y=100,c='r*')
    #ax.axis('off')
    trap_to_track_polysf.append(to_track_polysf)
    #trap_toTrack_cellIDf.append(to_track_cellIDsf)
    ax.set_title("%s %s"%(trap_list[ti],1))
    ax.set_aspect('equal')
    ax.invert_yaxis()

plt.tight_layout(pad=0,h_pad=0,w_pad=0)
    
    
    
    
    #fig.canvas.layout.width = '1280px'
    #fig.canvas.layout.height = '%spx'%(512*len(trap_tracked_cells))
    #fig.canvas.layout.width = '2000px'
    #fig.canvas.layout.height = '512px'
    #fig.canvas.layout.height = '%spx'%(512*len(trap_tracked_cells))
    #fig.canvas.layout.width = '500px'
    
    


cid = fig.canvas.mpl_connect('button_press_event', onclickSelectMulti)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

 ## Get cells path

In [ ]:
list_frames2=list_frames[-2:]
f1=[int(x) for x  in np.linspace(1,len(tracked_cells),len(tracked_cells))]
#list_frames2
list_frames2=f1

In [14]:

trap_path_list=[]

for i,trap in enumerate(trap_list):
    print(trap)
    cIDi=trap_toTrack_cellID0[i]
    cIDf=trap_toTrack_cellIDf[i]
    print(cIDi,cIDf)
    trap_df=trap_df_lineages[i]
    tyList=get_this_path(trap_df,cIDi,cIDf)
    trap_path_list.append(tyList)
len(trap_path_list[0])

xy03
['1.166', '1.162', '1.161', '1.165'] ['865.054', '865.056', '865.043', '865.051']
xy03-1
['1.014'] ['865.009']


4

## Plot paths

In [16]:
%matplotlib inline



for i,trap in enumerate(trap_list):
    tracked_cells=trap_tracked_cells[i]
    dirNameTRACKEDCELLS_IMGS=rootDir+'figures/traked_cells/'+trap+'/'
    print(dirNameTRACKEDCELLS_IMGS)
    paths=trap_path_list[i]
    
        
    for im,path in enumerate(paths):
        path_name="%s-%s"%(path[0],path[-1])
        dirNameIMAGEOVERLAY=dirNameTRACKEDCELLS_IMGS+'mothers_paths/path-%s/'%(path_name)
        
        if not os.path.exists(dirNameIMAGEOVERLAY):
            os.makedirs(dirNameIMAGEOVERLAY)
        
        for this_frame in list_frames:
            print(dirNameIMAGEOVERLAY," ",this_frame,end="\r")
            plt,matchID=plot_track_path(tracked_cells,this_frame,path,flims)
            #plt.title("%s %s path %s cellID %s"%(expeLabel,trap,path_name,matchID))
            framen=str(this_frame).zfill(3)
            figName="%s%s_%s_path-%s_%s_%s.png"%(dirNameIMAGEOVERLAY,expeLabel,trap,path_name,matchID,framen)
            plt.savefig(figName)
            plt.close("all")
        
        print()

../../data_sample/HT-LB/figures/traked_cells/xy03/
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_paths/path-1.166-865.054/   865
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_paths/path-1.162-865.056/   865
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_paths/path-1.161-865.043/   865
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_paths/path-1.165-865.051/   865 274
../../data_sample/HT-LB/figures/traked_cells/xy03-1/
../../data_sample/HT-LB/figures/traked_cells/xy03-1/mothers_paths/path-1.014-865.009/   865


## Mother Lineages

In [9]:


for i,trap in enumerate(trap_list):
    tracked_cells=trap_tracked_cells[i]
    df_lineages=trap_df_lineages_long[i]
    dirNameTRACKEDCELLS_IMGS=rootDir+'figures/traked_cells/'+trap+'/'
    print(dirNameTRACKEDCELLS_IMGS)
    
    
    this_frame=1
    frame_df=df_lineages[df_lineages['frame']==this_frame]
    mothers=frame_df.motherID.unique()
    mothers_lineage_ids=[]
    for this_mother in mothers:
        tdf=frame_df[frame_df["motherID"]==this_mother]
        this_lineageID=tdf.lineageID.unique()[0]
        mothers_lineage_ids.append(this_lineageID)
    print(mothers_lineage_ids)
    print(mothers)
    for im,this_lineage_id in enumerate(mothers_lineage_ids):
       # if(this_lineage_id!=1):
        #    continue
        this_frame=1
        #mother_name="%s.%s"%(this_frame,str(round(mothers[im]-this_frame,3)).ljust(5,'0').split(".")[1])
        mother_name=mothers[im]
        dirNameIMAGEOVERLAY=dirNameTRACKEDCELLS_IMGS+'mothers_lineages/lineage-%s_mother-%s/'%(this_lineage_id,mother_name)
        
        if not os.path.exists(dirNameIMAGEOVERLAY):
            os.makedirs(dirNameIMAGEOVERLAY)
        lineage_df=df_lineages[df_lineages['lineageID']==this_lineage_id]
        
        for this_frame in list_frames:
            print(dirNameIMAGEOVERLAY," ",this_frame,end="\r")
            frame_lineage_df=lineage_df[lineage_df['frame']==this_frame]
            this_daughters=frame_lineage_df["cellID"].tolist()
            #this_daughters=["%s.%s"%(this_frame,str(round(x-this_frame,3)).ljust(5,'0').split(".")[1]) for x in this_daughters]
            #print(this_daughters)
            #print(zzz)
            plt=plot_mother_lineages(tracked_cells,this_frame,this_daughters,flims)
            plt.title("%s %s mother %s lineage %s"%(expeLabel,trap,mother_name,this_lineage_id))
            framen=str(this_frame).zfill(3)
            figName="%s%s_%s_lineage-%s_mother-%s_%s.png"%(dirNameIMAGEOVERLAY,expeLabel,trap,this_lineage_id,mother_name,framen)
            plt.savefig(figName)
            plt.close("all")
        print()

../../data_sample/HT-LB/figures/traked_cells/xy03/
['1', '2', '3', '4', '5', '6', '7', '8']
['1.161' '1.141' '1.159' '1.162' '1.166' '1.148' '1.169' '1.165']
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_lineages/lineage-1_mother-1.161/   865
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_lineages/lineage-2_mother-1.141/   865
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_lineages/lineage-3_mother-1.159/   865
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_lineages/lineage-4_mother-1.162/   865
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_lineages/lineage-5_mother-1.166/   865
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_lineages/lineage-6_mother-1.148/   865278
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_lineages/lineage-7_mother-1.169/   865
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_lineages/lineage-8_mother-1.165/   865
../../data_sample/HT-LB/figures/traked_cells/xy03-1/
['1']
['1.

## Plot longest tracks

In [10]:


for i,trap in enumerate(trap_list):
    tracked_cells=trap_tracked_cells[i]
    df_lineages=trap_df_lineages_long[i]
    dirNameTRACKEDCELLS_IMGS=rootDir+'figures/traked_cells/'+trap+'/'
    print(dirNameTRACKEDCELLS_IMGS)
    
    
    this_frame=1
    frame_df=df_lineages[df_lineages['frame']==this_frame]
    mothers=frame_df.motherID.unique()
    mothers_lineage_ids=[]
    for this_mother in mothers:
        tdf=frame_df[frame_df["motherID"]==this_mother]
        this_lineageID=tdf.trackID.unique()[0]
        mothers_lineage_ids.append(this_lineageID)
    print(mothers_lineage_ids)
    
    for im,this_lineage_id in enumerate(mothers_lineage_ids):
       # if(this_lineage_id!=1):
        #    continue
        this_frame=1
        #mother_name="%s.%s"%(this_frame,str(round(mothers[im]-this_frame,3)).ljust(5,'0').split(".")[1])
        mother_name=mothers[im]
        dirNameIMAGEOVERLAY=dirNameTRACKEDCELLS_IMGS+'mothers_longestTracks_masks/mother-%s/'%(mother_name)
        
        if not os.path.exists(dirNameIMAGEOVERLAY):
            os.makedirs(dirNameIMAGEOVERLAY)
        lineage_df=df_lineages[df_lineages['trackID']==this_lineage_id]
        
        for this_frame in list_frames:
            print(dirNameIMAGEOVERLAY," ",this_frame,end="\r")
            frame_lineage_df=lineage_df[lineage_df['frame']==this_frame]
            this_daughters=frame_lineage_df["cellID"].tolist()
            #this_daughters=["%s.%s"%(this_frame,str(round(x-this_frame,3)).ljust(5,'0').split(".")[1]) for x in this_daughters]
            #print(this_daughters)
            #print(zzz)
            plt=plot_mother_lineages(tracked_cells,this_frame,this_daughters,flims)
            plt.title("%s %s mother %s track %s"%(expeLabel,trap,mother_name,this_lineage_id))
            framen=str(this_frame).zfill(3)
            figName="%s%s_%s_mother-%s_track-%s_%s.png"%(dirNameIMAGEOVERLAY,expeLabel,trap,mother_name,this_lineage_id,framen)
            plt.savefig(figName)
            plt.close("all")
            #plt.close()
        print()



../../data_sample/HT-LB/figures/traked_cells/xy03/
['1.161-865.000', '1.141-865.001', '1.159-865.002', '1.162-865.006', '1.166-865.009', '1.148-865.022', '1.169-865.028', '1.165-865.044']
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_longestTracks_masks/mother-1.161/   865
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_longestTracks_masks/mother-1.141/   8657331
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_longestTracks_masks/mother-1.159/   865
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_longestTracks_masks/mother-1.162/   865 246
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_longestTracks_masks/mother-1.166/   865
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_longestTracks_masks/mother-1.148/   865
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_longestTracks_masks/mother-1.169/   865193
../../data_sample/HT-LB/figures/traked_cells/xy03/mothers_longestTracks_masks/mother-1.165/   865
../../data_sample